In [1]:
import pandas as pd
import numpy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, zero_one_loss
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import glob

col_names = ["Flow ID","Source IP","Source Port","Destination IP","Destination Port","Protocol","Timestamp","Flow Duration","Total Fwd Packets","Total Backward Packets","Total Length of Fwd Packets","Total Length of Bwd Packets","Fwd Packet Length Max","Fwd Packet Length Min","Fwd Packet Length Mean","Fwd Packet Length Std","Bwd Packet Length Max","Bwd Packet Length Min","Bwd Packet Length Mean","Bwd Packet Length Std","Flow Bytes/s","Flow Packets/s","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Total","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max","Fwd IAT Min","Bwd IAT Total","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags","Fwd Header Length","Bwd Header Length","Fwd Packets/s","Bwd Packets/s","Min Packet Length","Max Packet Length","Packet Length Mean","Packet Length Std","Packet Length Variance","FIN Flag Count","SYN Flag Count","RST Flag Count","PSH Flag Count","ACK Flag Count","URG Flag Count","CWE Flag Count","ECE Flag Count","Down/Up Ratio","Average Packet Size","Avg Fwd Segment Size","Avg Bwd Segment Size","Fwd Header Length","Fwd Avg Bytes/Bulk","Fwd Avg Packets/Bulk","Fwd Avg Bulk Rate","Bwd Avg Bytes/Bulk","Bwd Avg Packets/Bulk","Bwd Avg Bulk Rate","Subflow Fwd Packets","Subflow Fwd Bytes","Subflow Bwd Packets","Subflow Bwd Bytes","Init_Win_bytes_forward","Init_Win_bytes_backward","act_data_pkt_fwd","min_seg_size_forward","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min","label"]

path = '/Users/xihajun/Documents/GSoC/OWASP-Seraphimdroid/data/Benign/Benign2017/' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, names = col_names)
    list_.append(df)

path = '/Users/xihajun/Documents/GSoC/OWASP-Seraphimdroid/data/Adware/'
allFiles = glob.glob(path + "*/*.csv")
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, names = col_names)
    list_.append(df)
    
frame = pd.concat(list_)

frame['label'].unique()

frame['label'] = frame['label'].replace(['ADWARE_SELFMITE', 'ADWARE_KOODOUS', 'ADWARE_KEMOGE',
       'ADWARE_GOOLIGAN', 'ADWARE_MOBIDASH', 'ADWARE_YOUMI',
       'ADWARE_DOWGIN', 'ADWARE_EWIND', 'ADWARE_FEIWO', 'ADWARE_SHUANET'], 'malware')

/Users/xihajun/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py:678: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


In [3]:
def categorical_converter(data):
    '''manually encode the three categorical columns into ordinals'''
    cols=["Flow ID","Source IP","Destination IP","Destination Port","Protocol","Timestamp",'label']
    # 1 is benign, 0 is malware

    for i in cols:
        data[i] = le.fit_transform(data[i])
    data = hot.fit_transform(data.loc[:,cols])
    
raw_data = frame

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder()
hot = OneHotEncoder(sparse=True, handle_unknown='ignore')
categorical_converter(raw_data)

In [5]:
# separate features (columns 1..40) and label (column 41)
features= raw_data.iloc[:,:raw_data.shape[1]-1]
labels= raw_data.iloc[:,raw_data.shape[1]-1:]

# convert them into numpy arrays
#features= numpy.array(features)
#labels= numpy.array(labels).ravel() # this becomes an 'horizontal' array
labels= labels.values.ravel() # this becomes a 'horizontal' array

# TODO: get features names and target name

# Separate data in train set and test set
df= pd.DataFrame(features)
# create training and testing vars
# Note: train_size + test_size < 1.0 means we are subsampling
# Use small numbers for slow classifiers, as KNN, Radius, SVC,...
X_train, X_test, y_train, y_test = train_test_split(df, labels, train_size=0.8, test_size=0.2)
print("X_train, y_train:", X_train.shape, y_train.shape)
print("X_test, y_test:", X_test.shape, y_test.shape)

X_train, y_train: (641016, 84) (641016,)
X_test, y_test: (160255, 84) (160255,)


In [6]:

# Training, choose model by commenting/uncommenting clf=
print("Training model")
clf= RandomForestClassifier(n_jobs=-1, random_state=3, n_estimators=102)#, max_features=0.8, min_samples_leaf=3, n_estimators=500, min_samples_split=3, random_state=10, verbose=1)
#clf = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, presort=False)

trained_model= clf.fit(X_train, y_train)

print("Score: ", trained_model.score(X_train, y_train))

Training model
Score:  1.0


In [7]:

# Predicting
y_pred = clf.predict(X_test)
y_pred

array([1, 1, 0, ..., 0, 0, 1])

In [8]:
results = confusion_matrix(y_test, y_pred)
error = zero_one_loss(y_test, y_pred)

print("Confusion matrix:\n", results)

Confusion matrix:
 [[81702   527]
 [  312 77714]]


In [9]:
print("Error: ", 1-error)


Error:  0.9947645939284265
